# LSTM을 이용한 텍스트 생성
- 데이터 다운로드 캐글https://www.kaggle.com/aashita/nyt-comments?select=ArticlesApril2018.csv
- ArticlesApril2018.csv

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
seed= 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]


Saving ArticlesApril2018.csv to ArticlesApril2018 (1).csv


In [4]:
import pandas as pd
df = pd.read_csv(filename, encoding='latin1')
df.head(2)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...


### 데이터 전처리

In [5]:
print('열의개수: ', len(df.columns))
print(df.columns)

열의개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
df['headline'].isnull().values.any()

False

In [7]:
headline = [text for text in df.headline.values]
headline[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
len(headline)

1324

In [9]:
# 노이즈데이터(Unknown)제거
headline = [text for text in headline if text !='Unknown']
len(headline)

1214

In [10]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
# 구두점 제거와 소문자화를 위한함수
def preprocessing(s):
  s=s.encode("utf8").decode("ascii", 'ignore')
  return ''.join(c for c in s if c not in punctuation).lower()

In [12]:
headlines = [preprocessing(x) for x in headline]
headlines[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [13]:
# 단어 집합(vocabulary)을 만들
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [14]:
sequences = []
for line in headlines:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
      sequence =encoded[:i+1]
      sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [15]:
index_to_word = {}
for key, value in t.word_index.items():
  index_to_word[value] = key

print('빈도수 상위 1번 단어 :' , index_to_word[1])
print('빈도수 상위 582번 단어 :', index_to_word[582])

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : offer


In [16]:
max_len = max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 24


In [17]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [18]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [19]:
Y = to_categorical(y, num_classes=vocab_size)

In [20]:
X.shape, Y.shape

((7803, 23), (7803, 3494))

### 모델 정의/설정/학습

In [21]:
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [22]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1, 
                    name="Embedding_Layer"))
model.add(LSTM(128, name="LSTM_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 556,834
Trainable params: 556,834
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [24]:
hitstory = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 4s - loss: 7.6493 - accuracy: 0.0300
Epoch 2/200
244/244 - 1s - loss: 7.1262 - accuracy: 0.0302
Epoch 3/200
244/244 - 1s - loss: 6.9862 - accuracy: 0.0313
Epoch 4/200
244/244 - 1s - loss: 6.8683 - accuracy: 0.0414
Epoch 5/200
244/244 - 1s - loss: 6.7320 - accuracy: 0.0437
Epoch 6/200
244/244 - 1s - loss: 6.5834 - accuracy: 0.0473
Epoch 7/200
244/244 - 1s - loss: 6.4172 - accuracy: 0.0470
Epoch 8/200
244/244 - 1s - loss: 6.2377 - accuracy: 0.0536
Epoch 9/200
244/244 - 1s - loss: 6.0490 - accuracy: 0.0578
Epoch 10/200
244/244 - 1s - loss: 5.8634 - accuracy: 0.0633
Epoch 11/200
244/244 - 1s - loss: 5.6900 - accuracy: 0.0686
Epoch 12/200
244/244 - 1s - loss: 5.5270 - accuracy: 0.0730
Epoch 13/200
244/244 - 1s - loss: 5.3665 - accuracy: 0.0787
Epoch 14/200
244/244 - 1s - loss: 5.2182 - accuracy: 0.0848
Epoch 15/200
244/244 - 1s - loss: 5.0705 - accuracy: 0.0948
Epoch 16/200
244/244 - 1s - loss: 4.9297 - accuracy: 0.1030
Epoch 17/200
244/244 - 1s - loss: 4.7957 - accura

In [25]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = np.argmax(model.predict(encoded), axis=-1)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [26]:
print(sentence_generation(model, t, 'i', 10))
print(sentence_generation(model, t, 'how', 10))
print(sentence_generation(model, t, 'former', 10))

i want to be rich and im not sorry case case
how do you feel about being told to smile just visit
former nfl cheerleaders settlement offer 1 and a meeting with goodell
